<a href="https://colab.research.google.com/github/wjunwei2001/TailorMade/blob/main/tiktok_llm_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate
# !pip install InstructorEmbedding
!pip install -U sentence-transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import re
import spacy
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import torch
import sentence_transformers
import accelerate
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.losses import ContrastiveTensionDataLoader
from datasets import Dataset

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('intfloat/e5-large-v2')
# input_texts = [
#     'query: how much protein should a female eat',
#     'query: summit define',
#     "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#     "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."
# ]
# embeddings = model.encode(input_texts, normalize_embeddings=True)

In [3]:
# Import SOTA text embedding model bsaed on the universally recognised Massive Text Embedding Benchmark (MTEB)

# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('EZlee/e-commerce-bert-base-multilingual-cased')
# model = SentenceTransformer("Salesforce/SFR-Embedding-2_R")
# model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
# dimensions = 512
# model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1", truncate_dim=dimensions)
# model = SentenceTransformer('Alibaba-NLP/gte-base-en-v1.5', trust_remote_code=True)
# model = SentenceTransformer("avsolatorio/GIST-Embedding-v0", revision=None)
model = SentenceTransformer('intfloat/e5-base-v2') # One of the top model for retrieval with instruction task

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

## LLM Finetuning (Self supervised)

In [4]:
df = pd.read_csv('amazon_data.csv')

In [5]:
df['description'].replace(np.nan, '', inplace=True)
# df['description'] = df['description'].apply(''.join)

In [6]:
df.head(5)

,title,original_price,sale_price,discount,rating,review_count,main_category,sub_category_1,sub_category_2,rankings,description,insights,purchase_cnt_prev_month,store_name,supplier,is_available,url
0,cycling sunglasses polarized men women sport s...,NaN,$7.99,NaN,3.3,3,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#52,093 in Sports & Outdoors (See Top 100 in S...",['【Flexible and Frame-less Design Frame】: The ...,NaN,NaN,ZJMZZM,ZJMZZM,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
1,"polarized sport sunglasses for men women, idea...",NaN,$19.99,NaN,4.5,38,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#270,555 in Sports & Outdoors (See Top 100 in ...",,NaN,NaN,ACBLUCE Store,ACBLUCE,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
2,"professional cycling glasses for men women, sp...",NaN,$8.49,NaN,4.6,54,Sports and Outdoors,Sports & Outdoors,Cycling Glasses & Goggles,"#2,851 in Sports & Outdoors (See Top 100 in Sp...",,"['Appearance_POSITIVE', 'Value_POSITIVE', 'Dar...",1K+ bought in past month,ROIBEAO,Roibeao Direct,In Stock,/sspa/click?ie=UTF8&spc=MToyMTgzOTMwMDMwOTIxMD...
3,rockbros bike bicycle cargo rack rear bike rac...,$43.99,$43.99,NaN,4.3,"2,803",Sports and Outdoors,Sports & Outdoors,Bike Cargo Racks,"#11,213 in Sports & Outdoors (See Top 100 in S...",['QUICK RELEASE CLAMP for easily mount on the ...,"['Quality_POSITIVE', 'Performance_POSITIVE', '...",700+ bought in past month,ROCKBROS Store,ROCKBROS LLC,In Stock,/RockBros-Bicycle-110-165-Capacity-Mountain/dp...
4,boao 20 pieces presta valve core bike replacem...,NaN,$9.99,NaN,4.6,832,Sports and Outdoors,Sports & Outdoors,Bike Tubes,"#9,574 in Sports & Outdoors (See Top 100 in Sp...",['Package include: 20 pieces presta valve core...,"['Quality_POSITIVE', 'Performance_POSITIVE', '...",600+ bought in past month,Boao Store,Hiroda US,In Stock,/Pieces-Replacement-Tubeless-Vittoria-Continen...


In [7]:
df['main_category'] = df['main_category'].str.lower()
df['main_category'].value_counts()

main_category
computers                   4363
home and kitchen            4230
electronics                 3854
beauty and personal care    3308
sports and outdoors         2853
baby products               2707
Name: count, dtype: int64

In [8]:
df['description'][0]

"['【Flexible and Frame-less Design Frame】: The newest Tr90 frame-less design for cycling glasses which make cycling sunglasses becoming more stylish and lightweight. The glasses leg is made of rubber which is flexible and comfortable.', '【UV400 Polarized Lenses】These Polarized Sport Sunglasses are took premium polarized lenses which are 100% UVA and UVB filtering, reduce reflections and glare for a clearer view. So your child can perform to the best of their ability in the Contest.', '【Ventilation Structure】: Our Sports Sunglasses have an excellent ventilation system with vents on the lenses and legs. This ventilation system not only prevents the lenses from fogging up, but also makes them more breathable when worn.', '【Designed for Sports】The Youth Sport Sunglasses are ideal for outdoor sports, such as baseball, softball, climbing, driving, fishing, golf, hiking, running, skiing, cycling and so on.', '【Anti Slip Design】: The legs of the polarized sunglasses are made of non-slip rubber

In [9]:
df['ecommerce_text'] = df['title'] + ' ' + df['description']

In [10]:
# Convert text to lowercase
df['ecommerce_text'] = df['ecommerce_text'].str.lower()

# Remove special characters and numbers
df['ecommerce_text'] = df['ecommerce_text'].apply(lambda x: re.sub('[^a-z\s]', '', str(x)))

In [11]:
df['ecommerce_text'][5]

'air pump replacement nozzles  pack inflatable pump nozzles head air pump inflator adaptor accessory for partial air bed air mattress etc  sizes it respectively in small medium and large size mouth easily meeting your needs to inflatable things of different mouth sizes made of quality plastic durable for long time using and lightweight for carrying in your travel bag and family toolbox choose the nozzle of proper size accordingly and install it on the air pump and then inflate the target object the small size one fit for small toys medium and large one fits for swimming pool boat inflatable bed this nozzles head is suitable for different kinds of pumps helping you to inflate swimming rings air bed air mattress inflatable pool sofa and more a good tool accessory for outdoor activities'

In [12]:
new_df = df.groupby('main_category').apply(lambda x: x.sample(min(len(x), 1000))).reset_index(drop=True)

# Verify the counts in the new DataFrame
print(new_df['main_category'].value_counts())

main_category
baby products               1000
beauty and personal care    1000
computers                   1000
electronics                 1000
home and kitchen            1000
sports and outdoors         1000
Name: count, dtype: int64


In [13]:
train_examples = new_df['ecommerce_text'].tolist()

In [14]:
train_dataloader = ContrastiveTensionDataLoader(train_examples, batch_size=8, pos_neg_ratio=8)
train_loss = losses.ContrastiveTensionLoss(model=model)

model.fit(
    [(train_dataloader, train_loss)],
    epochs=5,
)

Step,Training Loss
500,7.828500
1000,4.150900
1500,3.685000
2000,3.618300


In [15]:
!pip install huggingface_hub

In [16]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [17]:
model.push_to_hub("ecommerce_text_embedding_retrieval")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/wjunwei/ecommerce_text_embedding_retrieval/commit/7ff6635948be6ce8039e1970d53f937f521e4f33'

In [18]:
ecommerce_model = SentenceTransformer("wjunwei/ecommerce_text_embedding_retrieval")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/61.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

## Using the model for generating similarity score

In [25]:
# Do the same text processing for inference
df['title'] = df['title'].str.lower()
df['title'] = df['title'].apply(lambda x: re.sub('[^a-z\s]', '', str(x)))

df['product_embedding'] = 'passage: ' + df['ecommerce_text']
df['product_embedding'] = df['product_embedding'].apply(lambda x: ecommerce_model.encode(x))

In [48]:
df['ecommerce_text'][105]

'skull cap helmet liner beanie cooling mesh cycling running hat for men women fits under helmets black  black '

In [73]:
recipient_and_style_prompt = "query: Does this product meet the needs of a middle-aged man who is phiscally active?"

# user_input = "my uncle who loves to exercise"
# user_input2 = "my middle age wife"

In [74]:
user_input_embedding = model.encode(recipient_and_style_prompt, normalize_embeddings=True)
# user_input_embedding2 = model.encode(recipient_and_style_prompt, normalize_embeddings=True)

In [76]:
def calculate_similarity(embedding1, embedding2):
    return cosine_similarity([embedding1], [embedding2])

result1 = calculate_similarity(user_input_embedding, df['product_embedding'][105])
# result2 = calculate_similarity(user_input_embedding2, df['product_embedding'][105])

print(result1)
# print(result2)

[[-0.10104114]]


In [78]:
similarities = []

for product_embedding in df['product_embedding']:
    similarity = calculate_similarity(user_input_embedding, product_embedding)
    similarities.append(similarity)

# Add similarity results to the DataFrame
df['similarity_user'] = similarities

# Rank products based on similarity for each user input
df_sorted_user = df.sort_values(by='similarity_user', ascending=False)

# Display top few products for each user input
top_n = 5
print(f"Top {top_n} products:")
print(df_sorted_user[['title', 'similarity_user']].head(top_n))

Top 5 products:
                                                   title similarity_user
2080                             dakine surf and sun cap  [[0.41612872]]
1635                                   fjallraven kanken   [[0.4143922]]
20210                         gerber babygirls pair sock   [[0.2760872]]
2601                       the north face mudder trucker  [[0.23709384]]
20037  the childrens place baby boys pull on jogger s...   [[0.2020455]]
